In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from datetime import time

import os

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 0)

In [3]:
file = "diy_mco_21-May'25.csv"
out_file="diy_mco_data_20th May'25.csv"

In [4]:

raw_data=pd.read_csv(file,low_memory=False,memory_map=True)
data=pd.read_csv("format.csv",memory_map=True)
table_data=pd.read_csv("table_data.csv",memory_map=True)



In [5]:
raw_data.head(3)

,cust_id,lead_id,fse_cust_id,workflow_node_id,sub_stage,created_at,updated_at,solution_type_level_2,solution_type_level_3,entity_type,products_context,sbstage_count,name_match_status,mid,mid_creation_date,agent_team,qc_pending_max_created,panelqc_max_created,final_timestamp,first_qc_pending_min_created,sub_stage1
0,1082835970,97b8b377-e7d1-4950-a755-c48ae4c80fbb,1082835970,2136,PARTNER_PROFILE_EDIT_FAILURE,2025-03-30 05:19:02.000,2025-05-04 09:59:16.000,ybl_rekyc_update,NaN,INDIVIDUAL,NaN,NaN,True,weIXZn54666530775591,2023-10-05 17:41:12.000,NaN,NaN,NaN,2025-05-04 09:59:16.000,NaN,NaN
1,1364989554,2ad40e86-1505-4fec-a497-304982b56712,1364989554,156,LEAD_SUCCESSFULLY_CLOSED,2025-05-15 15:16:53.000,2025-05-15 15:20:57.000,ybl_rekyc_update,NaN,INDIVIDUAL,NaN,NaN,True,Sifcgd55192325315097,2022-02-18 08:42:06.000,NaN,NaN,NaN,NaN,NaN,NaN
2,666081567,9c5f55fe-24a4-4178-9394-e0794c009eaa,666081567,156,LEAD_SUCCESSFULLY_CLOSED,2025-05-14 00:21:40.000,2025-05-14 08:56:27.000,NaN,NaN,INDIVIDUAL,NaN,"{""PANEL_SUCCESS"":1}",True,jCTxSJ94279835497787,2025-05-14 08:51:03.000,NaN,2025-05-14 00:34:38.000,2025-05-14 08:50:56.000,2025-05-14 08:50:56.000,2025-05-14 00:34:38.000,NaN


In [6]:

raw_data.insert(5,'q',0)

In [7]:
table_map = table_data.set_index('NODE ID')['Lead Status'].to_dict()
raw_data['q'] = raw_data['workflow_node_id'].map(table_map)

raw_data['q'].unique()

array([nan, 'Merchant Live', 'Documents Rejected (Pending at Sales Team)',
       'Lead stuck in Intermediate Stages (Dedupe/Risk/PG rejections) ',
       'Lead Created/Posted (Pending at Sales team for  Document Upload)',
       'Leads Rejected in Risk Review'], dtype=object)

In [8]:
raw_data.loc[raw_data['sub_stage1']=='QC_AUTO_APPROVED','q']='Leads QC Auto Approved'

raw_data['q'].unique()

array([nan, 'Merchant Live', 'Documents Rejected (Pending at Sales Team)',
       'Leads QC Auto Approved',
       'Lead stuck in Intermediate Stages (Dedupe/Risk/PG rejections) ',
       'Lead Created/Posted (Pending at Sales team for  Document Upload)',
       'Leads Rejected in Risk Review'], dtype=object)

In [9]:
raw_data['sub_stage1'].value_counts()

sub_stage1
QC_AUTO_APPROVED    738
Name: count, dtype: int64

In [10]:
#raw_data.loc[raw_data['sub_stage1']=='QC_AUTO_APPROVED','q']='Leads QC Auto Approved'

In [11]:
raw_data=raw_data.dropna(subset=['q'])

raw_data['q'].value_counts()

q
Merchant Live                                                       63041
Documents Rejected (Pending at Sales Team)                          11465
Leads QC Auto Approved                                                738
Lead stuck in Intermediate Stages (Dedupe/Risk/PG rejections)         306
Lead Created/Posted (Pending at Sales team for  Document Upload)       56
Leads Rejected in Risk Review                                           1
Name: count, dtype: int64

In [12]:
raw_data['remove_date']=raw_data['updated_at'].copy()

raw_data['remove_date'] = pd.to_datetime(raw_data['remove_date'])

raw_data['remove_date'] = raw_data['remove_date'].dt.strftime('%Y-%m-%d')

raw_data['remove_date'] = pd.to_datetime(raw_data['remove_date'])


In [13]:

current_timestamp = pd.Timestamp.now().normalize()

raw_data=raw_data[raw_data['remove_date']!=current_timestamp]

In [14]:
raw_data=raw_data.drop(columns=['remove_date'])
len(raw_data)


73817

In [15]:
raw_data['solution_type_level_2'].unique()

array(['ybl_rekyc_update', nan, 'business_name_update', 'diy_upgrade',
       'billing_address_update', 'display_name_update',
       'auth_signatory_update', 'pan_addition',
       'registered_address_update', 'diy_edc', 'gst_update'], dtype=object)

In [16]:
raw_data=raw_data[raw_data['solution_type_level_2'].isin(['diy_upgrade','']) | raw_data['solution_type_level_2'].isna()]
len(raw_data)

33310

In [17]:
raw_data['solution_type_level_2'].unique()

array([nan, 'diy_upgrade'], dtype=object)

In [18]:
print(len(raw_data))
raw_data.head(3)

33310


,cust_id,lead_id,fse_cust_id,workflow_node_id,sub_stage,q,created_at,updated_at,solution_type_level_2,solution_type_level_3,entity_type,products_context,sbstage_count,name_match_status,mid,mid_creation_date,agent_team,qc_pending_max_created,panelqc_max_created,final_timestamp,first_qc_pending_min_created,sub_stage1
2,666081567,9c5f55fe-24a4-4178-9394-e0794c009eaa,666081567,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-05-14 00:21:40.000,2025-05-14 08:56:27.000,NaN,NaN,INDIVIDUAL,NaN,"{""PANEL_SUCCESS"":1}",True,jCTxSJ94279835497787,2025-05-14 08:51:03.000,NaN,2025-05-14 00:34:38.000,2025-05-14 08:50:56.000,2025-05-14 08:50:56.000,2025-05-14 00:34:38.000,NaN
7,1901811551,7f262a64-7715-4bd9-a72a-338c067c029b,1901811551,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-05-12 09:58:44.000,2025-05-12 10:16:41.000,NaN,NaN,INDIVIDUAL,NaN,"{""PANEL_SUCCESS"":1}",True,WFzmZQ79213420833945,2025-05-12 10:12:03.000,NaN,2025-05-12 10:07:29.000,2025-05-12 10:11:56.000,2025-05-12 10:11:56.000,2025-05-12 10:07:29.000,NaN
9,1951380315,d34ba893-6be9-4a67-b801-c32143cd65c9,1951380315,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-05-17 19:55:46.000,2025-05-17 20:21:37.000,NaN,NaN,INDIVIDUAL,NaN,"{""PANEL_SUCCESS"":1}",True,vQrSGS97752302201754,2025-05-17 20:12:59.000,NaN,2025-05-17 20:07:50.000,2025-05-17 20:12:50.000,2025-05-17 20:12:50.000,2025-05-17 20:07:50.000,NaN


In [19]:
raw_data.columns

Index(['cust_id', 'lead_id', 'fse_cust_id', 'workflow_node_id', 'sub_stage',
       'q', 'created_at', 'updated_at', 'solution_type_level_2',
       'solution_type_level_3', 'entity_type', 'products_context',
       'sbstage_count', 'name_match_status', 'mid', 'mid_creation_date',
       'agent_team', 'qc_pending_max_created', 'panelqc_max_created',
       'final_timestamp', 'first_qc_pending_min_created', 'sub_stage1'],
      dtype='object')

In [20]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node_id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3', 'solution_type'],
      dtype='object')

In [21]:
data[['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node_id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp',
       'Product_Context', 'Name_Match_Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp',
       'Stage movement count', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3','solution_type','first_qc_pending_min_created']]=raw_data[['cust_id', 'lead_id','entity_type','workflow_node_id', 'sub_stage',
       'q', 'created_at', 'updated_at','products_context','name_match_status','final_timestamp','panelqc_max_created','qc_pending_max_created','sbstage_count','fse_cust_id','agent_team','mid','mid_creation_date','solution_type_level_2','solution_type_level_3','first_qc_pending_min_created']]
       
len(data)

33310

In [22]:
data.head(3)


,Merchant Cust ID,Lead_ID,Entity_Type,Node_id,Lead_Stage,Lead Status,Created Timestamp,Updated Timestamp,Ageing,Product_Context,Name_Match_Status,10K/1rs MID Status,Final TimeStamp,QC Panel Successful TimeStamp,Last QC Pending Timestamp,TAT,Merchant QC TAT slab,Working TimeSlab,New Off Time,New Last QC Pending Time,Stage movement count,Rejection Ageing,cust_id,agent_team,mid,mid_created_date,solution_type_level_3,solution_type,first_qc_pending_min_created
2,666081567,9c5f55fe-24a4-4178-9394-e0794c009eaa,INDIVIDUAL,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-05-14 00:21:40.000,2025-05-14 08:56:27.000,NaN,NaN,True,NaN,2025-05-14 08:50:56.000,2025-05-14 08:50:56.000,2025-05-14 00:34:38.000,NaN,NaN,NaN,NaN,NaN,"{""PANEL_SUCCESS"":1}",NaN,666081567,NaN,jCTxSJ94279835497787,2025-05-14 08:51:03.000,NaN,NaN,2025-05-14 00:34:38.000
7,1901811551,7f262a64-7715-4bd9-a72a-338c067c029b,INDIVIDUAL,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-05-12 09:58:44.000,2025-05-12 10:16:41.000,NaN,NaN,True,NaN,2025-05-12 10:11:56.000,2025-05-12 10:11:56.000,2025-05-12 10:07:29.000,NaN,NaN,NaN,NaN,NaN,"{""PANEL_SUCCESS"":1}",NaN,1901811551,NaN,WFzmZQ79213420833945,2025-05-12 10:12:03.000,NaN,NaN,2025-05-12 10:07:29.000
9,1951380315,d34ba893-6be9-4a67-b801-c32143cd65c9,INDIVIDUAL,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-05-17 19:55:46.000,2025-05-17 20:21:37.000,NaN,NaN,True,NaN,2025-05-17 20:12:50.000,2025-05-17 20:12:50.000,2025-05-17 20:07:50.000,NaN,NaN,NaN,NaN,NaN,"{""PANEL_SUCCESS"":1}",NaN,1951380315,NaN,vQrSGS97752302201754,2025-05-17 20:12:59.000,NaN,NaN,2025-05-17 20:07:50.000


In [23]:
data['Updated Timestamp']=pd.to_datetime(data['Updated Timestamp'])
data['Created Timestamp']=pd.to_datetime(data['Created Timestamp'])
data['QC Panel Successful TimeStamp']=pd.to_datetime(data['QC Panel Successful TimeStamp'])


In [24]:
user_date = '2024-10-15'  # Example: YYYY-MM-DD format

# Convert the combined string into a datetime object
custom_timestamp = datetime.strptime(user_date, '%Y-%m-%d')

print(custom_timestamp)
print(type(custom_timestamp))

current_day = custom_timestamp.day
current_month = custom_timestamp.month
current_year = custom_timestamp.year

print(current_day)
print(current_month)
print(current_year)


2024-10-15 00:00:00
<class 'datetime.datetime'>
15
10
2024


In [25]:
current_date = datetime.now() - timedelta(days=7)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
print(current_date)

data.loc[(data['Updated Timestamp'].dt.day>=current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year), 'Ageing']='Last 7 days'

2025-05-14 13:54:32.201956


In [26]:
current_date = datetime.now() - timedelta(days=1)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
current_date

data.loc[(data['Updated Timestamp'].dt.day==current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year), 'Ageing']='Yesterday'

In [27]:
data['Ageing']=data['Ageing'].fillna('')

In [28]:
data['Ageing'].value_counts()

Ageing
               20863
Last 7 days    10564
Yesterday       1883
Name: count, dtype: int64

In [29]:


data.loc[data['Name_Match_Status']==True, ['Name_Match_Status', '10K/1rs MID Status']]=['Name Matched','10k MID created']

data.loc[data['Name_Match_Status']==False, ['Name_Match_Status', '10K/1rs MID Status']]=['Name Not Matched','1Rs MID created']

In [30]:
data['Name_Match_Status']=data['Name_Match_Status'].fillna('')

In [31]:
data['Name_Match_Status'].value_counts()

Name_Match_Status
Name Matched        28766
Name Not Matched     4544
Name: count, dtype: int64

In [32]:
data['Product_Context'].value_counts()

Series([], Name: count, dtype: int64)

In [33]:
data['mid_created_date']=pd.to_datetime(data['mid_created_date'],errors='coerce')

In [34]:
data['mid_created_date'].head(10)

2    2025-05-14 08:51:03
7    2025-05-12 10:12:03
9    2025-05-17 20:12:59
10   2021-01-27 14:05:19
12   2025-05-07 13:40:00
13   2025-05-16 08:31:52
14   2025-05-03 08:33:49
18   2025-05-19 14:08:20
28   2025-05-13 16:50:31
32   2025-05-08 17:28:00
Name: mid_created_date, dtype: datetime64[ns]

In [35]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node_id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3', 'solution_type',
       'first_qc_pending_min_created'],
      dtype='object')

In [36]:
current_date = datetime.now() - timedelta(days=7)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
print(current_date)

data.loc[ (data['Lead Status']== 'Documents Rejected (Pending at Sales Team)') & ((data['Updated Timestamp'].dt.day<current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year)), 'Rejection Ageing']='More Than 7 Days'

2025-05-14 13:54:32.386803


In [37]:
current_date = datetime.now() - timedelta(days=7)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
current_date

data.loc[ (data['Lead Status']== 'Documents Rejected (Pending at Sales Team)') & ((data['Updated Timestamp'].dt.day>=current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year)), 'Rejection Ageing']='6-7 Days'

In [38]:
current_date = datetime.now() - timedelta(days=5)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
current_date

data.loc[ (data['Lead Status']== 'Documents Rejected (Pending at Sales Team)') & ((data['Updated Timestamp'].dt.day>=current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year)), 'Rejection Ageing']='4-5 Days'

In [39]:
current_date = datetime.now() - timedelta(days=3)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
current_date

data.loc[ (data['Lead Status']== 'Documents Rejected (Pending at Sales Team)') & ((data['Updated Timestamp'].dt.day>=current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year)), 'Rejection Ageing']='2-3 Days'

In [40]:
current_date = datetime.now() - timedelta(days=1)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
current_date

data.loc[ (data['Lead Status']== 'Documents Rejected (Pending at Sales Team)') & ((data['Updated Timestamp'].dt.day==current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year)), 'Rejection Ageing']='Less Than 1 Day'

In [41]:
data['Rejection Ageing'].value_counts()

Rejection Ageing
More Than 7 Days    3669
4-5 Days             742
6-7 Days             723
2-3 Days             650
Less Than 1 Day      461
Name: count, dtype: int64

In [42]:
data['Rejection Ageing'].value_counts()

Rejection Ageing
More Than 7 Days    3669
4-5 Days             742
6-7 Days             723
2-3 Days             650
Less Than 1 Day      461
Name: count, dtype: int64

In [43]:
data.loc[~data['Entity_Type'].isin(['INDIVIDUAL','PROPRIETORSHIP']),'Entity_Type']='NON_PROPRIETORSHIP'

In [44]:
data['Entity_Type'].value_counts()

Entity_Type
INDIVIDUAL            30399
PROPRIETORSHIP         2900
NON_PROPRIETORSHIP       11
Name: count, dtype: int64

In [45]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node_id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3', 'solution_type',
       'first_qc_pending_min_created'],
      dtype='object')

In [46]:
data['Lead Status'].value_counts()

Lead Status
Merchant Live                                                       26295
Documents Rejected (Pending at Sales Team)                           6245
Leads QC Auto Approved                                                673
Lead stuck in Intermediate Stages (Dedupe/Risk/PG rejections)          96
Lead Created/Posted (Pending at Sales team for  Document Upload)        1
Name: count, dtype: int64

In [47]:
null_rejected_timeStamp=data[(data['Lead Status']=='Documents Rejected (Pending at Sales Team)')&(data['Last QC Pending Timestamp'].isnull())]


null_rejected_timeStamp=null_rejected_timeStamp[['Lead_ID']]

print(len(null_rejected_timeStamp))

null_rejected_timeStamp['id']="'" + null_rejected_timeStamp['Lead_ID'] + "'" + ","

null_rejected_timeStamp.head(3)

#null_rejected_timeStamp.to_csv("rejected_null.csv", index=False)


0


,Lead_ID,id


In [48]:
# raise Exception("Stop execution here")

print(len(data))

33310


In [49]:
#rejected_null_value=pd.read_csv('rejected_null_value.csv')

#rejected_null_value['q']=pd.to_datetime(rejected_null_value['q'],format='%d-%m-%Y %H:%M:%S')

#print(rejected_null_value.columns)

In [50]:
#print(rejected_null_value.head(3))
print(len(data))

33310


In [51]:

# Aggregate the right DataFrame by taking the first occurrence per lead_id
#rejected_agg = rejected_null_value.groupby('lead_id').first().reset_index()

# Merge the DataFrames on the single column
#data = data.merge(rejected_agg[['lead_id', 'Status']], left_on='Lead_ID', right_on='lead_id', how='left')

# Fill in values based on conditions
#data.loc[(data['Lead Status'] == 'Documents Rejected (Pending at Sales Team)') & (data['Last QC Pending Timestamp'].isnull()), 'Last QC Pending Timestamp'] = data['q']


In [52]:
#data=data.merge(rejected_null_value,left_on='Lead_ID',right_on='lead_id',how='left')

#data.loc[(data['Lead Status']=='Documents Rejected (Pending at Sales Team)') & (data['Last QC Pending Timestamp'].isnull()) ,'Last QC Pending Timestamp']=data['q']

#data=data.drop(columns=['lead_id','q'])

print(data.columns)
print(len(data[(data['Lead Status']=='Documents Rejected (Pending at Sales Team)')&(data['Last QC Pending Timestamp'].isnull())]))
print(len(data))

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node_id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3', 'solution_type',
       'first_qc_pending_min_created'],
      dtype='object')
0
33310


In [53]:
data['Final TimeStamp']=pd.to_datetime(data['Final TimeStamp'])
data['Last QC Pending Timestamp']=pd.to_datetime(data['Last QC Pending Timestamp'])

In [54]:
# type(data['Final TimeStamp'][0])
#print(type(data['Last QC Pending Timestamp'][0]))
print(len(data))

33310


In [55]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node_id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3', 'solution_type',
       'first_qc_pending_min_created'],
      dtype='object')

In [56]:
data['Last QC Pending Timestamp'].head(10)

2    2025-05-14 00:34:38
7    2025-05-12 10:07:29
9    2025-05-17 20:07:50
10   2025-05-20 15:28:21
12   2025-05-07 13:38:16
13   2025-05-15 22:59:08
14   2025-05-02 23:30:16
18   2025-05-19 14:03:14
28                   NaT
32   2025-05-08 17:26:02
Name: Last QC Pending Timestamp, dtype: datetime64[ns]

In [57]:


data['Last QC Pending Timestamp']=pd.to_datetime(data['Last QC Pending Timestamp'], errors='coerce')

# Define the start and end times
#start_time = time(8, 30, 0)
#end_time = time(21, 30, 0)

start_time = time(8, 00, 0)
end_time = time(22, 00, 0)



# Apply the condition to classify as 'Intime' or 'Offtime' based on the time component of 'N'
data['Working TimeSlab'] = data['Last QC Pending Timestamp'].apply(lambda x: 'Offtime' if pd.isna(x) else ('Intime' if start_time <= x.time() <= end_time else 'Offtime'))


data['Working TimeSlab'].head(10)

2     Offtime
7      Intime
9      Intime
10     Intime
12     Intime
13    Offtime
14    Offtime
18     Intime
28    Offtime
32     Intime
Name: Working TimeSlab, dtype: object

In [58]:
data['Working TimeSlab'].value_counts()

Working TimeSlab
Intime     27864
Offtime     5446
Name: count, dtype: int64

In [59]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node_id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3', 'solution_type',
       'first_qc_pending_min_created'],
      dtype='object')

In [60]:
#out_time_start = pd.to_datetime("21:30").time()
#out_time_end = pd.to_datetime("23:59:59").time()
#morning_time_end = pd.to_datetime("08:30").time()

out_time_start = pd.to_datetime("22:00").time()
out_time_end = pd.to_datetime("23:59:59").time()
morning_time_end = pd.to_datetime("08:00").time()

data['Last QC Pending Timestamp'] = pd.to_datetime(data['Last QC Pending Timestamp'], errors='coerce')


# Apply conditions to create column R
def process_timestamp(row):
    timestamp = row['Last QC Pending Timestamp']
    
    # Check if timestamp is NaT
    if pd.isna(timestamp):
        return np.nan
    
    time = timestamp.time()
    
    if row['Working TimeSlab'] == 'Offtime':
        if out_time_start <= time <= out_time_end:
            # Condition 1: Increment date by 1
            return timestamp + pd.DateOffset(days=1)
        elif time < morning_time_end:
            # Condition 2: Return the same timestamp
            return timestamp
    # Condition 3: Return blank for 'intime'
    return np.nan


data['New Off Time'] = data.apply(process_timestamp, axis=1)

In [61]:
# Function to create S column based on R
def generate_s(row):
    if pd.notna(row['New Off Time']):  # If R is not NaT or blank
        # Set time to 08:30 on the same date as R
        return row['New Off Time'].replace(hour=8, minute=00, second=0)
    else:
        # If R is NaT or blank, copy the value from Q
        return row['Last QC Pending Timestamp']
    

data['New Last QC Pending Time'] = data.apply(generate_s, axis=1)

In [62]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node_id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3', 'solution_type',
       'first_qc_pending_min_created'],
      dtype='object')

In [63]:
# Calculate the difference between L and S and format it as HH:mm:ss
def calculate_difference(row):
    if isinstance(row['New Last QC Pending Time'], pd.Timestamp) and isinstance(row['Final TimeStamp'], pd.Timestamp):
        diff = row['Final TimeStamp'] - row['New Last QC Pending Time']
        return diff  # Extract HH:mm:ss from Timedelta
    return np.nan  # Return NaN if either L or S is not a timestamp

# Apply function to create Difference column
data['TAT'] = data.apply(calculate_difference, axis=1)

In [64]:
data['TAT'].head(10)

2    0 days 00:50:56
7    0 days 00:04:27
9    0 days 00:05:00
10   0 days 00:10:33
12   0 days 00:01:33
13   0 days 00:31:49
14   0 days 00:33:45
18   0 days 00:04:57
28               NaT
32   0 days 00:01:50
Name: TAT, dtype: timedelta64[ns]

In [65]:
# Update TAT where it is negative to be the difference between L and N
def adjust_negative_tat(row):
    if pd.notna(row['TAT']) and row['TAT'] < pd.Timedelta(0):  # Check if TAT is negative
        if isinstance(row['Final TimeStamp'], pd.Timestamp) and isinstance(row['Last QC Pending Timestamp'], pd.Timestamp):
            return row['Final TimeStamp'] - row['Last QC Pending Timestamp']  # Calculate L - N if TAT is negative
    return row['TAT']  # Keep original TAT if not negative

# Apply function to adjust negative TAT values
data['TAT'] = data.apply(adjust_negative_tat, axis=1)

In [66]:
print(data[['Final TimeStamp','Last QC Pending Timestamp','TAT']].head(10))
print(len(data))

       Final TimeStamp Last QC Pending Timestamp             TAT
2  2025-05-14 08:50:56       2025-05-14 00:34:38 0 days 00:50:56
7  2025-05-12 10:11:56       2025-05-12 10:07:29 0 days 00:04:27
9  2025-05-17 20:12:50       2025-05-17 20:07:50 0 days 00:05:00
10 2025-05-20 15:38:54       2025-05-20 15:28:21 0 days 00:10:33
12 2025-05-07 13:39:49       2025-05-07 13:38:16 0 days 00:01:33
13 2025-05-16 08:31:49       2025-05-15 22:59:08 0 days 00:31:49
14 2025-05-03 08:33:45       2025-05-02 23:30:16 0 days 00:33:45
18 2025-05-19 14:08:11       2025-05-19 14:03:14 0 days 00:04:57
28                 NaT                       NaT             NaT
32 2025-05-08 17:27:52       2025-05-08 17:26:02 0 days 00:01:50
33310


In [67]:
#len(data[((data['Lead Status']=='Merchant Live') | (data['Lead Status']=='Documents Rejected (Pending at Sales Team)'))&(data['TAT'].isnull())])

data[((data['Lead Status']=='Merchant Live') | (data['Lead Status']=='Documents Rejected (Pending at Sales Team)'))&(data['TAT'].isnull())]

,Merchant Cust ID,Lead_ID,Entity_Type,Node_id,Lead_Stage,Lead Status,Created Timestamp,Updated Timestamp,Ageing,Product_Context,Name_Match_Status,10K/1rs MID Status,Final TimeStamp,QC Panel Successful TimeStamp,Last QC Pending Timestamp,TAT,Merchant QC TAT slab,Working TimeSlab,New Off Time,New Last QC Pending Time,Stage movement count,Rejection Ageing,cust_id,agent_team,mid,mid_created_date,solution_type_level_3,solution_type,first_qc_pending_min_created


In [68]:
len(data[~(((data['Lead Status'] == 'Merchant Live') | (data['Lead Status'] == 'Documents Rejected (Pending at Sales Team)')) & (data['TAT'].isnull()))])

33310

In [69]:
data = data[~(((data['Lead Status'] == 'Merchant Live') | (data['Lead Status'] == 'Documents Rejected (Pending at Sales Team)')) & (data['TAT'].isnull()))]


len(data[((data['Lead Status']=='Merchant Live') | (data['Lead Status']=='Documents Rejected (Pending at Sales Team)'))&(data['TAT'].isnull())])

0

In [70]:
def categorize_time(td):
    total_minutes = td.total_seconds() / 60  # Convert timedelta to total minutes
    if total_minutes < 2:
        return 'Leads QCed in less than 02 min'
    elif 2 <= total_minutes < 5:
        return 'Leads QCed in between 02 - 05 min'
    elif 5 <= total_minutes < 10:
        return 'Leads QCed in between 05 - 10 min'
    elif 10 <= total_minutes < 15:
        return 'Leads QCed in between 10 - 15 min'
    elif 15 <= total_minutes < 30:
        return 'Leads QCed in between 15 - 30 min'
    elif 30 <= total_minutes < 60:
        return 'Leads QCed in between 30 min - 01 hr'
    else:
        return 'leads QCed in more than 01 hr'



data.loc[(data['Lead Status']=='Merchant Live') | (data['Lead Status']=='Documents Rejected (Pending at Sales Team)'),'Merchant QC TAT slab']=data['TAT'].apply(categorize_time)



In [71]:
data['Merchant QC TAT slab'].value_counts()

Merchant QC TAT slab
Leads QCed in less than 02 min          10989
Leads QCed in between 02 - 05 min        7005
leads QCed in more than 01 hr            6578
Leads QCed in between 05 - 10 min        2836
Leads QCed in between 30 min - 01 hr     2441
Leads QCed in between 15 - 30 min        1743
Leads QCed in between 10 - 15 min         948
Name: count, dtype: int64

In [72]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node_id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3', 'solution_type',
       'first_qc_pending_min_created'],
      dtype='object')

In [73]:
data['solution_type_level_3'].unique()

array([nan, 'diy_upgrade'], dtype=object)

In [74]:
data.loc[(data['Entity_Type']=='INDIVIDUAL') & (data['solution_type_level_3'].isna()),'solution_type']='DIY_MCO'
data.loc[(data['Entity_Type']=='INDIVIDUAL') & (~data['solution_type_level_3'].isna()),'solution_type']='DIY_ReKYC'
data.loc[~(data['Entity_Type']=='INDIVIDUAL'),'solution_type']='DIY_Upgrade'

In [75]:
data['solution_type'].unique()

array(['DIY_MCO', 'DIY_Upgrade', 'DIY_ReKYC'], dtype=object)

In [76]:
# Convert date columns to datetime
data['first_qc_pending_min_created'] = pd.to_datetime(data['first_qc_pending_min_created'], errors='coerce')


In [ ]:
# Calculate the time difference (TAT) and handle NaT cases
data['end_to_end_tat'] = data['Final TimeStamp'] - data['first_qc_pending_min_created']
data['end_to_end_tat'] = data['end_to_end_tat'].apply(lambda x: '' if pd.isna(x) else str(x))

In [78]:

# Categorize the time difference
def categorize_time(tat):
    if tat == '':
        return ''  # Return blank for missing TAT
    # Convert string timedelta back to timedelta object
    td = pd.to_timedelta(tat)
    total_minutes = td.total_seconds() / 60  # Convert timedelta to minutes
    if total_minutes < 2:
        return 'Leads QCed in less than 02 min'
    elif 2 <= total_minutes < 5:
        return 'Leads QCed in between 02 - 05 min'
    elif 5 <= total_minutes < 10:
        return 'Leads QCed in between 05 - 10 min'
    elif 10 <= total_minutes < 15:
        return 'Leads QCed in between 10 - 15 min'
    elif 15 <= total_minutes < 30:
        return 'Leads QCed in between 15 - 30 min'
    elif 30 <= total_minutes < 60:
        return 'Leads QCed in between 30 min - 01 hr'
    else:
        return 'Leads QCed in more than 01 hr'

# Apply the function to the DataFrame based on the conditions
conditions = data['Lead Status'].isin(['Merchant Live', 'Documents Rejected (Pending at Sales Team)'])
data.loc[conditions, 'End to End TAT slab'] = data.loc[conditions, 'end_to_end_tat'].apply(categorize_time)

In [79]:
data.head(2)

,Merchant Cust ID,Lead_ID,Entity_Type,Node_id,Lead_Stage,Lead Status,Created Timestamp,Updated Timestamp,Ageing,Product_Context,Name_Match_Status,10K/1rs MID Status,Final TimeStamp,QC Panel Successful TimeStamp,Last QC Pending Timestamp,TAT,Merchant QC TAT slab,Working TimeSlab,New Off Time,New Last QC Pending Time,Stage movement count,Rejection Ageing,cust_id,agent_team,mid,mid_created_date,solution_type_level_3,solution_type,first_qc_pending_min_created,end_to_end_tat,End to End TAT slab
2,666081567,9c5f55fe-24a4-4178-9394-e0794c009eaa,INDIVIDUAL,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-05-14 00:21:40,2025-05-14 08:56:27,Last 7 days,NaN,Name Matched,10k MID created,2025-05-14 08:50:56,2025-05-14 08:50:56,2025-05-14 00:34:38,0 days 00:50:56,Leads QCed in between 30 min - 01 hr,Offtime,2025-05-14 00:34:38,2025-05-14 08:00:00,"{""PANEL_SUCCESS"":1}",NaN,666081567,NaN,jCTxSJ94279835497787,2025-05-14 08:51:03,NaN,DIY_MCO,2025-05-14 00:34:38,0 days 00:50:56,Leads QCed in between 30 min - 01 hr
7,1901811551,7f262a64-7715-4bd9-a72a-338c067c029b,INDIVIDUAL,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-05-12 09:58:44,2025-05-12 10:16:41,,NaN,Name Matched,10k MID created,2025-05-12 10:11:56,2025-05-12 10:11:56,2025-05-12 10:07:29,0 days 00:04:27,Leads QCed in between 02 - 05 min,Intime,NaT,2025-05-12 10:07:29,"{""PANEL_SUCCESS"":1}",NaN,1901811551,NaN,WFzmZQ79213420833945,2025-05-12 10:12:03,NaN,DIY_MCO,2025-05-12 10:07:29,0 days 00:04:27,Leads QCed in between 02 - 05 min


In [80]:
data.to_csv(out_file, index=False)

In [81]:
hzdjz

NameError: name 'hzdjz' is not defined

In [ ]:
#data.to_csv("all_flow_data_28th Oct'24.csv", index=False)

In [ ]:
sas

In [ ]:


# Convert to datetime
data['Last QC Pending Timestamp'] = pd.to_datetime(data['Last QC Pending Timestamp'])
data['Final TimeStamp'] = pd.to_datetime(data['Final TimeStamp'])

# Define active hours (08:30 AM - 09:30 PM)
active_start_time = pd.to_timedelta('08:30:00')
active_end_time = pd.to_timedelta('21:30:00')

# Function to calculate active duration for each row
def calculate_active_duration(row):
    start = row['Last QC Pending Timestamp']
    end = row['Final TimeStamp']
    
    # Check if either start or end is NaT
    if pd.isna(start) or pd.isna(end):
        return "00:00:00"  # Return 0 hours if either is NaT
    
    # Initialize total active duration
    total_active_duration = pd.Timedelta(0)
    
    # Iterate over each date from start to end
    for single_date in pd.date_range(start=start.date(), end=end.date()):
        active_start = pd.Timestamp(single_date) + active_start_time
        active_end = pd.Timestamp(single_date) + active_end_time
        
        # Determine the effective active period for this day
        effective_start = max(start, active_start)
        effective_end = min(end, active_end)

        # If there's a valid active period, add its duration
        if effective_start < effective_end:
            total_active_duration += effective_end - effective_start

    # Convert total active duration to hours
    total_hours = total_active_duration.total_seconds() / 3600
    
    # Format the hours to HH:MM:SS
    hours = int(total_hours)
    minutes = int((total_hours - hours) * 60)
    seconds = int(((total_hours - hours) * 60 - minutes) * 60)
    
    return f"{hours:02}:{minutes:02}:{seconds:02}"

# Apply the function to each row
data['TAT'] = data.apply(calculate_active_duration, axis=1)

# Display the result
print(data[['Last QC Pending Timestamp', 'Final TimeStamp', 'TAT']])
print(len(data))


In [ ]:
# data['TAT']=data['Final TimeStamp']-data['Last QC Pending Timestamp']
# data['TAT']=data['TAT'].apply(lambda x: str(x).split()[-1] if pd.notna(x) else 'NaT')
print(data.columns)
print(len(data))

In [ ]:
print(data[['Final TimeStamp','Last QC Pending Timestamp','TAT']].head(10))
print(len(data))

In [ ]:
len(data[((data['Lead Status']=='Merchant Live') | (data['Lead Status']=='Documents Rejected (Pending at Sales Team)'))&(data['TAT'].isnull())])

In [ ]:
data["TAT"]=pd.to_timedelta(data['TAT'])



print(data[['Final TimeStamp','Last QC Pending Timestamp','TAT']].head(10))

In [ ]:
type(data['TAT'][0])

In [ ]:
def categorize_time(td):
    total_minutes = td.total_seconds() / 60  # Convert timedelta to total minutes
    if total_minutes < 16:
        return 'Leads QCed in less than 15 min'
    elif 16 <= total_minutes < 36:
        return 'Leads QCed in between 15-30 min'
    elif 36 <= total_minutes < 66:
        return 'Leads QCed in between 30 min -1 hr'
    elif 66 <= total_minutes < 126:
        return 'Leads QCed in between 1 - 2 hr'
    elif 126 <= total_minutes < 186:
        return 'Leads QCed in between 2-3 hr'
    elif 186 <= total_minutes < 246:
        return 'Leads QCed in between 3-4 hr'
    else:
        return 'leads QCed in more than 4 hrs'



data.loc[(data['Lead Status']=='Merchant Live') | (data['Lead Status']=='Documents Rejected (Pending at Sales Team)'),'Merchant QC TAT slab']=data['TAT'].apply(categorize_time)


    



In [ ]:
print(data['Merchant QC TAT slab'].value_counts())
print(len(data))

In [ ]:
data['TAT'].head(10)

In [ ]:

# Function to convert timedelta to HH:MM:SS format with cumulative hours
def format_timedelta(td):
    total_seconds = int(td.total_seconds())
    hours, remainder = divmod(total_seconds, 3600)  # Convert total seconds to hours
    minutes, seconds = divmod(remainder, 60)
    return f'{hours:02}:{minutes:02}:{seconds:02}'  # Format as HH:MM:SS

# Apply the function to convert timedelta to HH:MM:SS
data['TAT'] = data['TAT'].apply(format_timedelta)

In [ ]:
data.loc[(data['Lead Status']!='Merchant Live') & (data['Lead Status']!='Documents Rejected (Pending at Sales Team)'),'TAT']=''

data['TAT'].head(10)

In [ ]:
data.columns

In [ ]:
data=data.drop(columns=['Working TimeSlab','New Off Time','New Last QC Pending Time','Node id'])
data.columns

In [ ]:
col=data.pop('TAT')

t=data.columns.get_loc('Merchant QC TAT slab')

data.insert(t+1,'TAT',col)

data.columns

In [ ]:
data['solution_type_level_3'].unique()
len(data)

In [ ]:
data_acup_kyc=data[data['solution_type_level_3'].isin(['ACCOUNT_UPGRADE', 'KYC_UPDATE'])]
data_mco=data[~data['solution_type_level_3'].isin(['ACCOUNT_UPGRADE', 'KYC_UPDATE'])]

In [ ]:
#raise Exception("Stop execution here")

In [ ]:
with pd.ExcelWriter("all_flow_data_21st Oct'24.xlsx") as writer:
    # Write the original dataframe to the first sheet
    data.to_excel(writer, sheet_name='All Data', index=False)
    
    # Write data filtered by Category 'A' and 'C' to the second sheet
    data_acup_kyc.to_excel(writer, sheet_name=' KYC & ACC UPGD', index=False)
    
    # Write data filtered by Category 'B' and 'D' to the third sheet
    data_mco.to_excel(writer, sheet_name='MCO', index=False)

In [ ]:
data.to_csv("all_flow_data_19th Oct'24.csv", index=False)
